# The Trading Hub Europe Gas Prediction Task


## General Setting

Natural gas has become an essential asset during the transition to renewable energies. As a commodity natural gas requires physical transport and corresponding storage facilities in order to meet the daily demands of consumers like households or industry. These demands need to be planned ahead to ensure that sufficient gas is available. In order to ensure a stable natural gas supply, the difference between input and offtake needs to be balanced on a daily basis. As the consumption of natural gas is highly variable, gas providers experience differences between the demand of gas and the amount of gas forecasted or contracted for. For Geramany, [Trading Hub Europe GmbH](https://www.tradinghub.eu/) has to account for this difference and ensure that for every day, the amount of consumed gas is equal to the amount of provided gas through buying (or selling) gas from (to) the market. Clearly balancing is associated with costs depending on the quality of the forecast of natural gas consumption and thus has an enormous optimisation potential.

## The Task

In this challenge, you will be provided with natural gas balancing data from Trading Hub Europe on a daily basis over a period from 2011 to 2021. The data will be organized into different folds, where the first fold consists of 150 days of training and 5 days for forecasting. Each subsequent fold will consist of 30 days for training and 5 for forecasting. For the training set, we will provide the net amount (margin) of gas bought or sold per day, aggregated for Germany. Your task will be to provide a multi-step-ahead forecast of the margin for each foreacast day.

We are concerned with two evaluation measures. The first is the Mean Absolute Error (MAE) across the first forecast day of each fold (i.e. the 1-day-ahead forecast error). The second is the MAE for the remaining 2-5 days. The final score to be submitted is the average of these two values. 


$$ 
score = \frac{0.5}{F}\left( \sum_{f} |f_{f,1}-y_{f,1}|\right) + \frac{0.5}{4F} \left(\sum_{f}\sum_{t=2}^{5}|f_{f,t}-y_{f,t}|\right)
$$

where $f_{f,t}$ is your provided forecast for the $f$-th fold and $t$-th forecast day in a fold and $y_{f,t}$ is the true net amount of gas to be bought / sold for the $f$-th fold and $t$-th forecast in a fold day. $F$ is the total number of folds. 


You are allowed to use external data like for example weather data, but no other data from the natural gas market or future data in the provided dataset (i.e. data beyond the period you want to forecast for). 

<p>From 1st of October 2011 onwards we provide a full year of data. After this first 365, we provide intervals of 30 days for training and 5 days as forecasting target (see the evaluation intervalls below as well as the example submission file.).
You are allowed to re-use past forecasts, but you are not allowed to use any future information in the data (or external) beyond the forecasting period.</p>

## Phases of the Challenge

The challenge is organised in two phases:

<p> <b>Phase 1 - Model Development:</b></p> In phase 1 we provide you the training data, which you can use to develop your models. Submission to the system will be evaluated on a subset of the ground-truth and only using an unweighted mean absolute error on the forecasts. This is done in order to avoid optimization on the ground truth through repeated submissions. The number of submissions will also be limited to 100.

<p><b>Phase 2 - Evaluation:</b></p> In phase 2 we will evaluate your submissions on the true test set. You will be allowed to conduct 20 submissions, whereas the last submission will be counted as final score. 

# The Data

The data is provided in the file `train.csv` with one line of data per day, with the first line containing the header data. The following columns are available:

- `day` day in the form YYYY-MM-DD
- `SLP` expected consumption in natural gas networks obtained by small costumers obatined via standard profiles ([Standardlastprofile](https://de.wikipedia.org/wiki/Standardlastprofil)). Unit is KWH
- `RLM` measured consumption in natural gas for larger industries (Registrierte Leistungsmessung). Unit is KWH. Will be not available for days in the forecasting range and one day before. 
- `ENTRY` real energy (Einspeisung) provided to the network in KWH. Will be not available for days in the forecasting range and one day before. 
- `EXIT` real energy in MWH consumed (Ausspeisung) by small costumers. Will be not available for days in the forecasting range and one day before. 
- `BK-Saldo` difference in forcasted consumption from the real consumption for large consumers (mostly industries) (Bilanzkreislaufsaldo). Will be not available for days in the forecasting range.  
- `NK-Saldo` difference in forecasting of the consumption from the real consumption for small consumer (e.g. households). Will be not available for days in the forecasting range. 
- `holiday` 1 if the day is a german holiday and 0 otherwise 
- `SystemBuy` amount of energy to be bought, Will be not available for days in the forecasting range and one day before.
- `SystemSell` amount of energy to be sold, Will be not available for days in the forecasting range and one day before. 
- `margin` difference between energy which needed to be bought vs energy to be sold (i.e. `SystemBuy-SystemSell`). Forecasting target, which will be not available for days in the forecasting range. 


Note that approximately `SLP+RLM+EXITSO_ALL = ENTRYSO_ALL` holds but due to the underlying physical characteristics of the network, there is no explicit relation. The acutal phyiscal energy is decoupled from the market energy and needs to be balanced on a daily basis. So `BK-Saldo` and `NK-Saldo` hint towards the daily difference in physical energy aggregated over larger consumers (e.g. industrie) and smaler consumers (e.g. households, city gas networks). Due to the different properties of the phyiscal networks (e.g. storage, size) there is no direct, explicitly expressable equation in the data for the balancing `margin` and the other variables.

In [2]:
# The following code loads the training data in to a pandas frame
import pandas as pd
df_train = pd.read_csv("./train.csv", sep=";", decimal=".")
df_train.index = df_train["day"]
df_train.drop(columns=["day"], inplace=True)
df_train.head()

BK-Saldo   SystemSell  SystemBuy         ENTRY          EXIT  \
day                                                                           
2011-10-01  185578966.0  191914000.0        0.0  3.291493e+09  1.813804e+09   
2011-10-02  214244658.0  120660000.0        0.0  3.314660e+09  1.887688e+09   
2011-10-03  293628343.0  169770000.0        0.0  3.338293e+09  1.725050e+09   
2011-10-04   70450930.0  137430000.0        0.0  3.545043e+09  1.876402e+09   
2011-10-05   52963565.0  126892000.0        0.0  3.540875e+09  1.807170e+09   

                     RLM        SLP    NK-Saldo  holiday       margin  
day                                                                    
2011-10-01  1.153378e+09  311631504  38843506.0        0 -191914000.0  
2011-10-02  1.035181e+09  312944496  54787355.0        0 -120660000.0  
2011-10-03  1.147403e+09  307815984  49116718.0        1 -169770000.0  
2011-10-04  1.415471e+09  317136888  16499263.0        0 -137430000.0  
2011-10-05  1.467581e+09  347090088 -12899181.0        0 -126892000.0

### Auxilliary Weather Data

We also provide weather data for every day in the file "de-weather-data-aggregated.csv.gz". The data is aggregated over all weather stations in Germany (provided by the DWD) using [Meteostat](https://dev.meteostat.net/python/daily.html#data-structure)

- `tavg_mean` Mean daily average temperature in Celcius over all weather stations 
- `tavg_std` standard deviation of daily average temperature in Celcius over all weather stations
- `tmin_mean` Mean daily minimum temperature in Celcius over all weather stations 
- `tmin_std` standard deviation of daily minimum temperature in Celcius over all weather stations
- `tmax_mean` Mean daily maximum temperature in Celcius over all weather stations 
- `tmax_std` standard deviation of daily maximum temperature in Celcius over all weather stations
- `prcp_mean` Mean daily precipitation in mm over all weather stations 
- `prcp_std` standard deviation of daily precipitation in mm over all weather stations
- `snow_mean` Mean daily snow in mm over all weather stations 
- `snow_std` standard deviation of daily snow in mm over all weather stations
- `wdir_mean` Mean daily average wind direction in degrees over all weather stations 
- `wdir_std` standard deviation of daily average wind direction in degrees over all weather stations
- `wspd_mean` Mean daily average wind speed in km/h over all weather stations 
- `wspd_std` standard deviation of daily average wind speed in km/h over all weather stations
- `wpgt_mean` Mean daily peak wind gust in km/h over all weather stations 
- `wpgt_std` standard deviation of daily peak wind gust in km/h over all weather stations
- `pres_mean` Mean daily average  sea-level air pressure in hPa over all weather stations 
- `pres_std` standard deviation of daily average sea-level air pressure in hPa over all weather stations
- `tsun_mean` Mean daily daily sunshine total in minutes (m) over all weather stations 
- `tsun_std` standard deviation of daily daily sunshine total in minutes (m) over all weather stations


In [4]:
weather = pd.read_csv("./de-weather-data-aggregated.csv.gz", compression="gzip")
weather.index = weather["time"]
weather.drop(["time"], inplace=True, axis=1)
weather.head()

tavg_mean  tavg_std  tmin_mean  tmin_std  tmax_mean  tmax_std  \
time                                                                        
2011-10-01  15.972024  1.478982   9.010861  2.276562  24.801844  2.162091   
2011-10-02  15.799802  1.457334   9.058487  2.312000  24.462577  2.041137   
2011-10-03  16.016110  1.526839   9.727049  2.674383  23.255943  1.936191   
2011-10-04  16.212181  1.705390  11.268852  2.805080  21.993033  2.662191   
2011-10-05  15.239020  1.613860  12.259796  2.417742  18.277143  1.898069   

            prcp_mean  prcp_std  snow_mean  snow_std   wdir_mean   wdir_std  \
time                                                                          
2011-10-01   0.000930  0.009602   0.107296  2.316205  138.163077  78.731278   
2011-10-02   0.000826  0.009053   0.085837  1.852964  180.375897  79.650038   
2011-10-03   0.012744  0.067601   0.000000  0.000000  215.590306  48.946977   
2011-10-04   0.035457  0.121745   0.000000  0.000000  234.016583  31.766763   
2011-10-05   0.282734  1.022187   0.000000  0.000000  237.420690  23.914716   

            wspd_mean  wspd_std  wpgt_mean   wpgt_std    pres_mean  pres_std  \
time                                                                           
2011-10-01   5.465778  2.742609  17.970874   5.235910  1026.485714  1.041294   
2011-10-02   5.394222  2.838308  18.304831   6.103634  1024.568421  1.177170   
2011-10-03   9.659912  6.057572  27.531579   9.676059  1020.908095  2.613652   
2011-10-04  14.707018  7.999160  39.784038  11.418692  1019.124519  3.500093   
2011-10-05  15.577391  8.521225  39.436321  12.012133  1018.720853  3.856922   

             tsun_mean    tsun_std  
time                                
2011-10-01  630.434483   77.779683  
2011-10-02  608.309278   72.849210  
2011-10-03  463.582192  192.576678  
2011-10-04  417.112628  202.281586  
2011-10-05   64.767123  111.045085

# Submission of Results

The submission is a csv file named `submission.csv` compressed in a zip file with one line per forecast containing the day and the forecasting value, as well as a header line 
containing the column names "day" for the forecasting day and "margin" for the forecasted value for that day. The format looks like:

```
day;margin
2015-07-29; 5000
```

where day falls into the following time ranges (including  start and end date )

```
2012-10-25 00:00:00 - 2012-10-29 00:00:00
2012-11-24 00:00:00 - 2012-11-28 00:00:00
2012-12-24 00:00:00 - 2012-12-28 00:00:00
2013-01-23 00:00:00 - 2013-01-27 00:00:00
2013-02-22 00:00:00 - 2013-02-26 00:00:00
2013-03-24 00:00:00 - 2013-03-28 00:00:00
2013-04-23 00:00:00 - 2013-04-27 00:00:00
2013-05-23 00:00:00 - 2013-05-27 00:00:00
2013-06-22 00:00:00 - 2013-06-26 00:00:00
2013-07-22 00:00:00 - 2013-07-26 00:00:00
2013-08-21 00:00:00 - 2013-08-25 00:00:00
2013-09-20 00:00:00 - 2013-09-24 00:00:00
2013-10-20 00:00:00 - 2013-10-24 00:00:00
2013-11-19 00:00:00 - 2013-11-23 00:00:00
2013-12-19 00:00:00 - 2013-12-23 00:00:00
2014-01-18 00:00:00 - 2014-01-22 00:00:00
2014-02-17 00:00:00 - 2014-02-21 00:00:00
2014-03-19 00:00:00 - 2014-03-23 00:00:00
2014-04-18 00:00:00 - 2014-04-22 00:00:00
2014-05-18 00:00:00 - 2014-05-22 00:00:00
2014-06-17 00:00:00 - 2014-06-21 00:00:00
2014-07-17 00:00:00 - 2014-07-21 00:00:00
2014-08-16 00:00:00 - 2014-08-20 00:00:00
2014-09-15 00:00:00 - 2014-09-19 00:00:00
2014-10-15 00:00:00 - 2014-10-19 00:00:00
2014-11-14 00:00:00 - 2014-11-18 00:00:00
2014-12-14 00:00:00 - 2014-12-18 00:00:00
2015-01-13 00:00:00 - 2015-01-17 00:00:00
2015-02-12 00:00:00 - 2015-02-16 00:00:00
2015-03-14 00:00:00 - 2015-03-18 00:00:00
2015-04-13 00:00:00 - 2015-04-17 00:00:00
2015-05-13 00:00:00 - 2015-05-17 00:00:00
2015-06-12 00:00:00 - 2015-06-16 00:00:00
2015-07-12 00:00:00 - 2015-07-16 00:00:00
2015-08-11 00:00:00 - 2015-08-15 00:00:00
2015-09-10 00:00:00 - 2015-09-14 00:00:00
2015-10-10 00:00:00 - 2015-10-14 00:00:00
2015-11-09 00:00:00 - 2015-11-13 00:00:00
2015-12-09 00:00:00 - 2015-12-13 00:00:00
2016-01-08 00:00:00 - 2016-01-12 00:00:00
2016-02-07 00:00:00 - 2016-02-11 00:00:00
2016-03-08 00:00:00 - 2016-03-12 00:00:00
2016-04-07 00:00:00 - 2016-04-11 00:00:00
2016-05-07 00:00:00 - 2016-05-11 00:00:00
2016-06-06 00:00:00 - 2016-06-10 00:00:00
2016-07-06 00:00:00 - 2016-07-10 00:00:00
2016-08-05 00:00:00 - 2016-08-09 00:00:00
2016-09-04 00:00:00 - 2016-09-08 00:00:00
2016-10-04 00:00:00 - 2016-10-08 00:00:00
2016-11-03 00:00:00 - 2016-11-07 00:00:00
2016-12-03 00:00:00 - 2016-12-07 00:00:00
2017-01-02 00:00:00 - 2017-01-06 00:00:00
2017-02-01 00:00:00 - 2017-02-05 00:00:00
2017-03-03 00:00:00 - 2017-03-07 00:00:00
2017-04-02 00:00:00 - 2017-04-06 00:00:00
2017-05-02 00:00:00 - 2017-05-06 00:00:00
2017-06-01 00:00:00 - 2017-06-05 00:00:00
2017-07-01 00:00:00 - 2017-07-05 00:00:00
2017-07-31 00:00:00 - 2017-08-04 00:00:00
2017-08-30 00:00:00 - 2017-09-03 00:00:00
2017-09-29 00:00:00 - 2017-10-03 00:00:00
2017-10-29 00:00:00 - 2017-11-02 00:00:00
2017-11-28 00:00:00 - 2017-12-02 00:00:00
2017-12-28 00:00:00 - 2018-01-01 00:00:00
2018-01-27 00:00:00 - 2018-01-31 00:00:00
2018-02-26 00:00:00 - 2018-03-02 00:00:00
2018-03-28 00:00:00 - 2018-04-01 00:00:00
2018-04-27 00:00:00 - 2018-05-01 00:00:00
2018-05-27 00:00:00 - 2018-05-31 00:00:00
2018-06-26 00:00:00 - 2018-06-30 00:00:00
2018-07-26 00:00:00 - 2018-07-30 00:00:00
2018-08-25 00:00:00 - 2018-08-29 00:00:00
2018-09-24 00:00:00 - 2018-09-28 00:00:00
2018-10-24 00:00:00 - 2018-10-28 00:00:00
2018-11-23 00:00:00 - 2018-11-27 00:00:00
2018-12-23 00:00:00 - 2018-12-27 00:00:00
2019-01-22 00:00:00 - 2019-01-26 00:00:00
2019-02-21 00:00:00 - 2019-02-25 00:00:00
2019-03-23 00:00:00 - 2019-03-27 00:00:00
2019-04-22 00:00:00 - 2019-04-26 00:00:00
2019-05-22 00:00:00 - 2019-05-26 00:00:00
2019-06-21 00:00:00 - 2019-06-25 00:00:00
2019-07-21 00:00:00 - 2019-07-25 00:00:00
2019-08-20 00:00:00 - 2019-08-24 00:00:00
2019-09-19 00:00:00 - 2019-09-23 00:00:00
2019-10-19 00:00:00 - 2019-10-23 00:00:00
2019-11-18 00:00:00 - 2019-11-22 00:00:00
2019-12-18 00:00:00 - 2019-12-22 00:00:00
2020-01-17 00:00:00 - 2020-01-21 00:00:00
2020-02-16 00:00:00 - 2020-02-20 00:00:00
2020-03-17 00:00:00 - 2020-03-21 00:00:00
2020-04-16 00:00:00 - 2020-04-20 00:00:00
2020-05-16 00:00:00 - 2020-05-20 00:00:00
2020-06-15 00:00:00 - 2020-06-19 00:00:00
2020-07-15 00:00:00 - 2020-07-19 00:00:00
2020-08-14 00:00:00 - 2020-08-18 00:00:00
2020-09-13 00:00:00 - 2020-09-17 00:00:00
2020-10-13 00:00:00 - 2020-10-17 00:00:00
2020-11-12 00:00:00 - 2020-11-16 00:00:00
2020-12-12 00:00:00 - 2020-12-16 00:00:00
2021-01-11 00:00:00 - 2021-01-15 00:00:00
2021-02-10 00:00:00 - 2021-02-14 00:00:00
2021-03-12 00:00:00 - 2021-03-16 00:00:00
2021-04-11 00:00:00 - 2021-04-15 00:00:00
2021-05-11 00:00:00 - 2021-05-15 00:00:00
2021-06-10 00:00:00 - 2021-06-14 00:00:00
2021-07-10 00:00:00 - 2021-07-14 00:00:00
2021-08-09 00:00:00 - 2021-08-13 00:00:00
2021-09-08 00:00:00 - 2021-09-12 00:00:00
```

Note that the file must be zipped. So you either put it into a zip archive or you zip the file itself with the code below.

In [12]:
# The following code loads the sample submission format and makes a primitive forecast
import pandas as pd
df_submit = pd.read_csv("./submission_example.csv", sep=";", decimal=".")
df_submit.index=df_submit["day"]
df_submit.head()
df_submit["margin"]=df_train["margin"].median()
df_submit.to_csv("./submission.zip", sep=";", decimal=".", compression=dict(method='zip',
                        archive_name='submission.csv') ) #note that you need to compress it as zip which contains a submission.csv file

## Evaluation

The following code briefly shows the evaluation from the hidden ground truth file for the trivial forecasting above. 

In [13]:
# The following code loads the sample submission format and evaluates it to the primitive forecast from above. 
# The code resembles the evaluation procedure on the server, but is not identically. gt.csv is obviously not provided
import pandas as pd
df_gt = pd.read_csv("./gt.csv", sep=";", decimal=".")
df_gt.index=df_gt["day"]
df_gt.head()
five_ahead = (df_submit["margin"]-df_gt["margin"]).abs().mean()
one_ahead_ix = df_submit.index[0::5]
one_ahead = (df_submit["margin"].loc[one_ahead_ix] -df_gt["margin"].loc[one_ahead_ix]).abs().mean()
print(f"1-day-ahead {one_ahead} + 5-day-ahead {five_ahead}: Total score = {0.5*one_ahead+0.5*five_ahead}")

1-day-ahead 77509385.32110092 + 5-day-ahead 82353330.27522936: Total score = 79931357.79816514


In [16]:
df_submit2 = pd.read_csv("./submission.csv", sep=";", decimal=".", compression="zip")
df_gt2 = pd.read_csv("./ground_truth.csv", sep=";", decimal=".")
df_merged = pd.merge(df_gt2.iloc[0:150],df_submit2,how="left", on="day", suffixes=("_gt","_submission"))
df_merged


day  margin_gt       day.1  margin_submission
0    2012-10-25  185676000  2012-10-25                0.0
1    2012-10-26   79290000  2012-10-26                0.0
2    2012-10-27   78062000  2012-10-27                0.0
3    2012-10-28   65280000  2012-10-28                0.0
4    2012-10-29  202560000  2012-10-29                0.0
..          ...        ...         ...                ...
145  2015-03-14   -5123000  2015-03-14                0.0
146  2015-03-15  -54994000  2015-03-15                0.0
147  2015-03-16  -50037000  2015-03-16                0.0
148  2015-03-17 -105807000  2015-03-17                0.0
149  2015-03-18  -69660000  2015-03-18                0.0

[150 rows x 4 columns]

In [10]:
df_gt2

day     margin
0    2012-10-25  185676000
1    2012-10-26   79290000
2    2012-10-27   78062000
3    2012-10-28   65280000
4    2012-10-29  202560000
..          ...        ...
540  2021-09-08    8882000
541  2021-09-09   62769000
542  2021-09-10   65180000
543  2021-09-11   32515000
544  2021-09-12  -15405000

[545 rows x 2 columns]